<a href="https://colab.research.google.com/github/sanjanagarai/InterIIT_Team1_Task2/blob/main/VisualBERT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Importing modules


In [ ]:
import pandas as pd

import torch
from torchvision.models.detection import fasterrcnn_resnet50_fpn
from torchvision.transforms import functional as F
from PIL import Image
from tqdm import tqdm
from pathlib import Path
import torch
from torchvision import transforms as T
import torch.nn as nn

from transformers import BertTokenizer
from transformers import BertTokenizer, VisualBertForPreTraining

## IMAGE FEATURE EXTRACTION

Loading the fasterrcnn_resnet model


In [ ]:
# Load pre-trained Faster R-CNN model with ResNet-50 backbone
model = fasterrcnn_resnet50_fpn(pretrained=True)
model.eval()

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=FasterRCNN_ResNet50_FPN_Weights.COCO_V1`. You can also use `weights=FasterRCNN_ResNet50_FPN_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/fasterrcnn_resnet50_fpn_coco-258fb6c6.pth" to /root/.cache/torch/hub/checkpoints/fasterrcnn_resnet50_fpn_coco-258fb6c6.pth
100%|██████████| 160M/160M [00:00<00:00, 227MB/s]


FasterRCNN(
  (transform): GeneralizedRCNNTransform(
      Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
      Resize(min_size=(800,), max_size=1333, mode='bilinear')
  )
  (backbone): BackboneWithFPN(
    (body): IntermediateLayerGetter(
      (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (bn1): FrozenBatchNorm2d(64, eps=0.0)
      (relu): ReLU(inplace=True)
      (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
      (layer1): Sequential(
        (0): Bottleneck(
          (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn1): FrozenBatchNorm2d(64, eps=0.0)
          (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn2): FrozenBatchNorm2d(64, eps=0.0)
          (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn3): FrozenBatchNorm2d(256, eps=0.0)
          (relu): ReLU(

connecting to drive

In [ ]:
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')


Mounted at /content/drive


Creating object_features_tensor from the images by extracting features from the images

In [ ]:
# Create a Path object for the image folder
image_folder_path = Path("/content/drive/MyDrive/VQA_RAD_Dataset/RGB_images")

# List to store object feature tensors
object_feature_tensors = []

# Initialize the transform to convert images to tensors and resize them
transform = T.Compose([T.Resize((224, 224)), T.ToTensor()])

# Iterate through each image in the folder
for image_path in tqdm(image_folder_path.glob("*.jpg")):
    # Load image
    img = Image.open(image_path).convert("RGB")

    # Perform inference and extract object features (boxes)
    with torch.no_grad():
        predictions = model(transform(img).unsqueeze(0))[0]

    if "boxes" in predictions:
        boxes = predictions["boxes"]
        for box in boxes:
            x1, y1, x2, y2 = box.tolist()

            # Extract object region from the image and resize to a fixed size
            object_img = img.crop((x1, y1, x2, y2))
            object_img_resized = transform(object_img).unsqueeze(0)

            # Append object feature tensor to the list
            object_feature_tensors.append(object_img_resized)


9it [00:39,  4.41s/it]


KeyboardInterrupt: ignored

In [ ]:
# Concatenate object feature tensors along the batch dimension
concatenated_features = torch.cat(object_feature_tensors, dim=0)

In [ ]:
print(concatenated_features.shape)

torch.Size([4680, 3, 224, 224])


Defining the custom cnn model to obtain the visual_embed tensor from the object_features_tensor

In [ ]:
# Define a custom CNN model for visual embeddings
class CustomCNN(nn.Module):
    def __init__(self, input_channels, output_channels):
        super(CustomCNN, self).__init__()
        self.conv1 = nn.Conv2d(input_channels, 64, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1)
        self.conv3 = nn.Conv2d(128, output_channels, kernel_size=3, stride=1, padding=1)
        self.relu = nn.ReLU()
        self.maxpool = nn.MaxPool2d(kernel_size=2, stride=2)

    def forward(self, x):
        x = self.relu(self.conv1(x))
        x = self.maxpool(x)
        x = self.relu(self.conv2(x))
        x = self.maxpool(x)
        x = self.relu(self.conv3(x))
        x = self.maxpool(x)
        return x

In [ ]:
# Initialize the custom CNN model for visual embeddings
custom_cnn_model = CustomCNN(input_channels=3, output_channels=512)

# Pass the concatenated features through the custom CNN model to obtain visual embeddings
visual_embeds = custom_cnn_model(concatenated_features)

# Now, visual_embed has the shape (num_objects, 512) representing visual embeddings for each object
print("Visual Embed Shape:", visual_embeds.shape)

## QUESTION FEATURE EXTRACTION

In [ ]:
visual_embeds=concatenated_features

In [ ]:
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 25.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 36.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 52.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 62.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 31.5 MB/s eta 0:00:00


Question and answer tokens are obtained using BERT

In [ ]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

In [ ]:
print(visual_embeds.shape)

torch.Size([4680, 3, 224, 224])


In [ ]:
# Read the CSV file with a specific encoding
df = pd.read_csv('/content/question_data_final.csv', encoding='ISO-8859-1')

# Select the required columns: image_name, question, and answer
new_df = df[['image_name', 'question', 'answer']]

# Save the updated dataset back to q&a_space.csv
new_df.to_csv('q&a_space.csv', index=False)


In [ ]:
print(len(questions))

2248


In [ ]:
list=[]
for i in range(0,2248):
  list.append(questions[i])

In [ ]:
print(list)

['Are regions of the brain infarcted?', 'Are the lungs normal appearing?', 'Is there evidence of a pneumothorax', 'What type of imaging does this not represent?', 'Is this a MRI of the chest?', 'What is not pictured in this image?', 'Is the trachea midline?', 'Is there evidence of an aortic aneurysm?', 'Where is the abnormality?', 'Is there blunting of the costovertebral angles?', 'Is there evidence of an aortic aneurysm?', 'Which organ system is abnormal in this image?', 'Is there airspace consolidation on the left side?', 'Is this an axial plane?', 'Where is the pathology in this image?', 'Is there a pneumothorax?', 'What type of imaging is this?', 'What organ system is pictured?', 'Is there a fracture?', 'How is the patient oriented?', 'Is there swelling of the grey matter?', 'Is there vascular pathology in this image?', 'Is this an anterior posterior image?', 'Is there cardiomyopathy?', 'Is this the brain?', 'Is there air under the diaphragm?', 'What type of image is this?', 'Is th

In [ ]:
questions=df['question']
tokens = tokenizer(questions[0], padding='max_length', max_length=50)

## Fusion Layer

Using VisualBERT for fusion

In [ ]:
input_ids = torch.tensor(tokens["input_ids"])
attention_mask = torch.tensor(tokens["attention_mask"])
token_type_ids = torch.tensor(tokens["token_type_ids"])

In [ ]:
model = VisualBertForPreTraining.from_pretrained('uclanlp/visualbert-nlvr2-coco-pre') # this checkpoint has 1024 dimensional visual embeddings projection

In [ ]:
# Assuming visual_embeds is already a tensor with shape [4680, 3, 224, 224]
visual_attention_mask = torch.ones(visual_embeds.shape[:-1], dtype=torch.long)
visual_token_type_ids = torch.ones(visual_embeds.shape[:-1], dtype=torch.long)


In [ ]:
print(input_ids.shape)
print(attention_mask.shape)
print(token_type_ids.shape)
print(visual_embeds.shape)
print(visual_attention_mask.shape)
print(visual_token_type_ids.shape)

torch.Size([50])
torch.Size([50])
torch.Size([50])
torch.Size([4680, 3, 224, 224])
torch.Size([4680, 3, 224])
torch.Size([4680, 3, 224])


In [ ]:
# Assuming you have the tensors as you described
input_ids = input_ids.unsqueeze(1)  # Add sequence length dimension: [50] -> [50, 1]
attention_mask = attention_mask.unsqueeze(1)  # Add sequence length dimension: [50] -> [50, 1]
token_type_ids = token_type_ids.unsqueeze(1)  # Add sequence length dimension: [50] -> [50, 1]

# Assuming visual_attention_mask and visual_token_type_ids need to be adjusted
visual_attention_mask = visual_attention_mask[:, :, :1]  # Adjust dimensions, keep only the first element
visual_token_type_ids = visual_token_type_ids[:, :, :1]  # Adjust dimensions, keep only the first element

In [ ]:
print(input_ids.shape)
print(attention_mask.shape)
print(token_type_ids.shape)
print(visual_embeds.shape)
print(visual_attention_mask.shape)
print(visual_token_type_ids.shape)

torch.Size([50, 1, 1])
torch.Size([50, 1, 1])
torch.Size([50, 1, 1])
torch.Size([4680, 3, 224, 224])
torch.Size([4680, 3, 1])
torch.Size([4680, 3, 1])


In [ ]:
# Assuming you have the tensors as you described
input_ids = input_ids.view(-1, 1)  # Reshape to [50, 1]
attention_mask = attention_mask.view(-1, 1)  # Reshape to [50, 1]
token_type_ids = token_type_ids.view(-1, 1)  # Reshape to [50, 1]

# Flatten the visual_embeds tensor
visual_embeds = visual_embeds.view(4680, -1)  # Reshape to [4680, 3 * 224 * 224]

# Flatten the visual_attention_mask and visual_token_type_ids tensors
visual_attention_mask = visual_attention_mask.view(4680, -1)  # Reshape to [4680, 3 * 224]
visual_token_type_ids = visual_token_type_ids.view(4680, -1)  # Reshape to [4680, 3 * 224]


In [ ]:
print(input_ids.shape)
print(attention_mask.shape)
print(token_type_ids.shape)
print(visual_embeds.shape)
print(visual_attention_mask.shape)
print(visual_token_type_ids.shape)

torch.Size([50, 1])
torch.Size([50, 1])
torch.Size([50, 1])
torch.Size([4680, 150528])
torch.Size([4680, 3])
torch.Size([4680, 3])


training code

In [ ]:
outputs = model
 (input_ids=input_ids,
  attention_mask=attention_mask,
  token_type_ids=token_type_ids,
  visual_embeds=visual_embeds,
  visual_attention_mask=visual_attention_mask
  , visual_token_type_ids=visual_token_type_ids)

RuntimeError: ignored

In [ ]:
outputs